In [ ]:
# versions: pandas==2.0.3, lifelines==0.27.8, nibabel==5.2.1, numpy==1.24.4, scipy==1.8.0, statsmodels==0.14.1,scikit-learn==1.3.2

In [ ]:
import pandas as pd
from lifelines import CoxPHFitter
import numpy as np
import statsmodels.stats.multitest as smt
import nibabel as nib
from sklearn.preprocessing import RobustScaler

In [ ]:
DATASET_TRAIN="./data/CN+MCI_BL+VOL_train.csv"
DATASET_TEST="./data/CN+MCI_BL+VOL_test.csv"

In [ ]:
FASTSURFER_OUTPUT_DIR="./data/ADNI/output_fastsurfer/"
VBM_RESULTS_DIR="./results/VBM/ADNI/"

In [ ]:
traindf=pd.read_csv(DATASET_TRAIN)
testdf=pd.read_csv(DATASET_TEST)

In [ ]:
unc_p_values=[]
for i in range(0,1749353):
    df=pd.read_csv(VBM_RESULTS_DIR+"/FastSurfer_Results_TimeToEvent_all_Untersuchung_TrainValidTest_pixel_"+str(i)+".csv",header=None)
    df.columns=["pixel","age","gender","time","status","fieldstrength","eTIV","PTID","imageuid"]
    train=df[df.imageuid.isin(traindf.IMAGEUID)]
    df=df[df.imageuid.isin(testdf.IMAGEUID)]
    df=df.drop_duplicates("imageuid")
    train=train.drop_duplicates("imageuid")

    df.loc[df["fieldstrength"]=="1.5 T","fieldstrength"]=0
    df.loc[df["fieldstrength"]=="3.0 T","fieldstrength"]=1
    train.loc[train["fieldstrength"]=="1.5 T","fieldstrength"]=0
    train.loc[train["fieldstrength"]=="3.0 T","fieldstrength"]=1
    df=df.drop("imageuid",axis=1)
    df=df.drop("PTID",axis=1)
    train=train.drop("imageuid",axis=1)
    train=train.drop("PTID",axis=1)

    df=df.astype({"fieldstrength": "int64"})
    train=train.astype({"fieldstrength": "int64"})

    if(i%10000==0):
        print("Pixel "+str(i)+" of "+ str(1749353)+ " ("+str(i/1749353)+" %)")
    df["status_corr"]=df.status.isin(["pCN","pMCI"])
    train["status_corr"]=train.status.isin(["pCN","pMCI"])
    df=df.drop("status",axis=1)
    train=train.drop("status",axis=1)
    scaler = RobustScaler().fit(train.drop(["time","status_corr"],axis=1))
    df_pre=scaler.transform(df.drop(["time","status_corr"],axis=1))
    df[df.drop(["time","status_corr"],axis=1).columns]=df_pre
    if(df.pixel.var()==0.0):
        unc_p_values.append(1.0)
    else:
        cph = CoxPHFitter(penalizer=0.1)
        cph.fit(df, "time", "status_corr" )
        p_values = cph.summary["p"]
        unc_p_values.append(p_values["pixel"])

In [ ]:
img=np.reshape(unc_p_values, (113,137, 113))
img=(img-1)*(-1)
new_image = nib.Nifti1Image(img, affine=np.eye(4))

In [ ]:
nib.save(new_image, VBM_RESULTS_DIR+"/VBM_p_value_results_uncorrected.nii.gz")  

In [ ]:
res=smt.multipletests(unc_p_values, alpha=0.01, method="fdr_bh", is_sorted=False, returnsorted=False)
img=np.reshape(res[1].tolist(), (113,137, 113))

In [ ]:
img=(img-1)*(-1)

In [ ]:
new_image = nib.Nifti1Image(img, affine=np.eye(4))

In [ ]:
nib.save(new_image, VBM_RESULTS_DIR+"/VBM_p_value_results_all_fdr.nii.gz")  